In [ ]:
#importing libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from dataprep.eda import plot
from dataprep.eda import plot_correlation, plot_missing
from dataprep.eda import *
import plotly.express as px
import plotly.figure_factory as ff
from collections import Counter
import warnings
from sklearn.model_selection import train_test_split
warnings.filterwarnings("ignore")

In [ ]:
df = pd.read_csv('stroke_prediction.csv')

pd.set_option('display.max_columns', None)

print(df.head())

In [ ]:
#Drop id column

df.drop(columns=['id'],inplace=True) 

print(df.head())

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.isna


In [ ]:
df.isnull().sum()

In [ ]:
plot_missing(df)

In [ ]:
df.drop(df.index[df['gender'] == 'Other'], inplace = True)

In [ ]:
#imputing the missing values using mean

df=df.fillna(np.mean(df['bmi']))
df.info()

In [ ]:
import pandas_profiling as pp
pp.ProfileReport(df)

In [ ]:
plot(df)


In [ ]:
df

In [ ]:
plot(df, 'bmi')


In [ ]:
plot_correlation(df)


In [ ]:


create_report(df)



In [ ]:
sns.countplot(x="stroke", data=df)
plt.show()

In [ ]:


plt.figure(figsize=(15,8))
sns.heatmap(df.corr())

In [ ]:


# Removing the observations that have smoking_status type unknown. 
df=df[df['smoking_status']!='Unknown']



In [ ]:
df.columns


In [ ]:
df['work_type'].replace(['Self-employed'], 'Self_employed')

In [ ]:
df

In [ ]:
df.corr()

In [ ]:
df = pd.get_dummies(data= df, columns= ['gender', 'work_type', 'smoking_status'], drop_first= True)

In [ ]:
df

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

df['ever_married'] = le.fit_transform(df['ever_married'])
df['Residence_type'] = le.fit_transform(df['Residence_type'])

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
x = df.drop('stroke', axis = 1)
y = df['stroke']


In [ ]:
x.columns

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x,y,test_size=0.20,random_state=0)

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)


In [ ]:
from imblearn.over_sampling import SMOTE
from collections import Counter
counter = Counter(y_train)
print('Before', counter)
smt = SMOTE()
X_train_sm, y_train_sm = smt.fit_sample(X_train, y_train.ravel())
counter = Counter(y_train_sm)
print('After', counter)



In [ ]:
from sklearn import metrics 
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.ensemble import RandomForestClassifier

In [ ]:
clf = RandomForestClassifier()

In [ ]:
clf.fit(X_train_sm, y_train_sm)

In [ ]:
predictions = clf.predict(X_test)

In [ ]:
print(metrics.classification_report(y_test, predictions))

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score

In [ ]:
print(accuracy_score(y_test, predictions)*100)

In [ ]:
print(confusion_matrix(y_test, predictions))

In [ ]:
import numpy as np
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 500, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 1000,10)]
# Minimum number of samples required to split a node
min_samples_split = [2, 5]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
              'criterion':['entropy','gini']}
print(random_grid)

In [ ]:
rf=RandomForestClassifier()
rf_randomcv=RandomizedSearchCV(estimator=rf,param_distributions=random_grid,n_iter=100,cv=3,verbose=2,
                               random_state=100,n_jobs=-1)
### fit the randomized model
rf_randomcv.fit(X_train_sm, y_train_sm)

In [ ]:
rf_randomcv.best_params_

In [ ]:
rf_randomcv

In [ ]:
best_random_grid=rf_randomcv.best_estimator_

In [ ]:
from sklearn.metrics import accuracy_score
y_pred=best_random_grid.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print("Accuracy Score {}".format(accuracy_score(y_test,y_pred)))
print("Classification report: {}".format(classification_report(y_test,y_pred)))

In [ ]:
import pickle
# open a file, where you ant to store the data
file = open('RFC.pkl', 'wb')

# dump information to that file
pickle.dump(best_random_grid, file)